# Federated Learning on small GPT model

In [1]:
!pip install -q 'flwr[simulation]' torch torchvision matplotlib

In [2]:
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torchvision
# import torchvision.transforms as tfms
# from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader, random_split
import string
import pandas as pd

import flwr as fl
from flwr.common import Metrics
from tqdm import tqdm

DEVICE = torch.device(
    "cuda" if torch.backends.cuda.is_built()
    else "mps" if torch.backends.mps.is_built()
    else "cpu"
)
# DEVICE = "cpu"

print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flwr {fl.__version__}")

Training on cuda using PyTorch 2.1.0+cu118 and Flwr 1.5.0


In [3]:
df = pd.read_csv('en-fr-mid.csv')
df.head()

,Unnamed: 0,en,fr,en_len,fr_len
0,0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...,201,248
1,1,Site map,Plan du site,8,12
2,2,Feedback,Rétroaction,8,11
3,3,Credits,Crédits,7,7
4,4,Français,English,8,7


In [4]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)
# df = df.head(5000)
df['en_len'] = df['en'].apply(lambda x: len(x))
df['fr_len'] = df['fr'].apply(lambda x: len(x))
df.shape

(19999, 5)

In [5]:
df[['en_len', 'fr_len']].describe()

,en_len,fr_len
count,19999.000000,19999.000000
mean,117.508825,141.033802
std,108.681920,126.543994
min,1.000000,1.000000
25%,62.000000,74.000000
50%,104.000000,124.000000
75%,152.000000,184.000000
max,5821.000000,6159.000000


In [6]:
# CLASSES = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck",)
BATCH_SIZE = 32
NUM_CLIENTS = 3

vocab = list(set(' '.join(df['en'].values.tolist() + df['fr'].values.tolist())))
vocab = ["<s>", "</s>", "<pad>"] + vocab
ix2ch = {ix:ch for ix,ch in enumerate(vocab)}
ch2ix = {ch:ix for ix,ch in ix2ch.items()}
encode = lambda s: [ch2ix[c] for c in s]
decode = lambda l: ''.join([ix2ch[i] for i in l])


class NanoGptDataset(Dataset):
    def __init__(self, texts: List[str]) -> None:
        super().__init__()
        self.texts = texts

    def __len__(self) -> int:
        return len(self.texts)

    def __getitem__(self, ix: int):
        text = self.texts[ix]
#         text = ''.join([i if ord(i) < 128 else ' ' for i in text.strip])
        input_ids = [ch2ix['<s>']] + encode(text) # [<s> a b c d   e ]
        input_ids = input_ids[:256]
        output_ids = input_ids[1:] + [ch2ix['</s>']] # [ a  b c d e </s>]
        assert len(input_ids) == len(output_ids), print(input_ids, output_ids, "\n\n======= Something went wrong when encoding the input and outputs ========\n\n")
        return  {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'labels': torch.tensor(output_ids, dtype=torch.long)
        }


def collate_fn(batch):
    max_len = 0
    for b in batch:
        max_len = max(len(b['input_ids']), max_len)
#         print({k:v.shape for k, v in b.items()})

    res = None

    for b in batch:
        req_padding = max_len - len(b['input_ids'])
        if res is None:
            if req_padding == 0:
                res = {k:v[None, ...] for k,v in b.items()}
            else:
                res = {
                    'input_ids': torch.hstack([b['input_ids'], torch.tensor([ch2ix['<pad>']]*req_padding, dtype=torch.long)])[None, ...],
                    'labels': torch.hstack([b['labels'],  torch.tensor([ch2ix['<pad>']]*req_padding, dtype=torch.long)])[None, ...]
                }
            continue

        if res is not None:
            if req_padding == 0:
                res = {
                    k: torch.cat([res[k], b[k].view(1, max_len)], dim=0) for k,v in res.items()
                }
            else:
                tmp = {
                    'input_ids': torch.hstack([b['input_ids'], torch.tensor([ch2ix['<pad>']]*req_padding, dtype=torch.long)])[None, ...],
                    'labels': torch.hstack([b['labels'], torch.tensor([ch2ix['<pad>']]*req_padding, dtype=torch.long)])[None, ...]
                }
                res = {
                    k: torch.cat([res[k], tmp[k].view(1, max_len)], dim=0) for k,v in res.items()
                }
    return res


def load_datasets(train_texts: List[str], test_texts: List[str], num_clients: int):
    trainset = NanoGptDataset(texts=train_texts)
    testset = NanoGptDataset(texts=test_texts)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(
        trainset, lengths=lengths, generator=torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    validloaders = []
    for ds in datasets:
        len_val = len(ds) // 10
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        print(lengths)
        ds_train, ds_val = random_split(
            ds, lengths=lengths, generator=torch.Generator().manual_seed(42)
        )
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn))
        validloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, collate_fn=collate_fn))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    return trainloaders, validloaders, testloader

In [7]:
texts = df['en'].values.tolist() + df['fr'].values.tolist()
import random
random.shuffle(texts)

train_texts = texts[:18000]
test_texts = texts[18000:]

trainloaders, validloaders, testloader = load_datasets(train_texts=train_texts, test_texts=test_texts, num_clients=NUM_CLIENTS)

[5400, 600]
[5400, 600]
[5400, 600]


## Step 1: Centralized Training with PyTorch

### Define a model

In [8]:
import math
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F


# ---------------------------------- Config ---------------------------------- #
@dataclass
class GPTConfig:
    buffer_size: int = 256
    vocab_size: int =  len(vocab) # 50304  # GPT2 has a total of 50257, padded to nearest multiple of 64 for efficiency
    n_layers: int = 3
    n_head: int = 4
    n_embed: int = 368
    dropout: float = 0.1
    bias: bool = False
    use_sinusoidal: bool = True


# ----------------------------- Attention Module ----------------------------- #
class Attention(nn.Module):
    '''Unlike RNNs where we were required to get one output and then pass it back onto the RNN and repeat the process
    again and again, here with masked attention, we simply find the lower triangular matrix and then weight them according
    the vector product the lower triangular matrix and the embedding vectors, we  build a masked representation for each word only using
    the values which occured/was predicted prior to the current index.
        - This is achieved by the torch.tril function and masking all zeros to -torch.inf (taking softmax makes it equal to zero)
    '''
    def __init__(self, n_embed: int, head_size: int) -> None:
        super().__init__()
        self.Q = nn.Linear(n_embed, head_size, bias=GPTConfig.bias)
        self.K = nn.Linear(n_embed, head_size, bias=GPTConfig.bias)
        self.V = nn.Linear(n_embed, head_size, bias=GPTConfig.bias)
        tril = torch.tril(
            torch.ones(size=(GPTConfig.buffer_size, GPTConfig.buffer_size))
        )
        self.register_buffer("tril", tril)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        _, T, _ = x.shape
        q, k, v = self.Q(x), self.K(x), self.V(x)  # (B, T, C) => (B, T, H)
        wei = (
            q @ k.mT * (1.0 / math.sqrt(k.size(-1)))
        )  # (B, T, H) @ (B, H, T) = (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        return wei @ v  # (B, T, T) @ (B, T, H) => (B, T, H)


# --------------------------- Multi Head Attention --------------------------- #
class MultiHeadAttention(nn.Module):
    def __init__(self, n_embed: int, n_heads: int) -> None:
        super().__init__()
        assert (
            n_embed % n_heads == 0
        ), "The number of heads must divide the embedding dimensions"
        head_size = n_embed // n_heads
        self.heads = nn.ModuleList(
            [Attention(n_embed=n_embed, head_size=head_size) for _ in range(n_heads)]
        )
        self.proj = nn.Linear(n_embed, n_embed, bias=GPTConfig.bias)
        self.dropout = nn.Dropout(p=GPTConfig.dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, C) -> (B, T, C//N_HEADS) -> (B, T, C)
        return self.dropout(self.proj(x)) #  (B, T, C) -> (B, T, C)


# ------------------------------- Feed Forward ------------------------------- #
class FeedForward(nn.Module):
    def __init__(self, n_embed: int) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed), nn.GELU(), nn.Linear(4 * n_embed, n_embed)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


# ------------------------------ Attention Block ----------------------------- #
class AttentionBlock(nn.Module):
    def __init__(self, n_embed: int, n_heads: int) -> None:
        super().__init__()
        self.sa = MultiHeadAttention(n_embed=n_embed, n_heads=n_heads)
        self.ffwd = FeedForward(n_embed=n_embed)
        self.ln1 = nn.LayerNorm((n_embed,))
        self.ln2 = nn.LayerNorm((n_embed,))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.sa(self.ln1(x)) # (B, T, C) -> (B, T, C)
        x = x + self.ffwd(self.ln2(x))
        return x


# --------------------------- Positional Embeddings -------------------------- #
class PositionalEncoding(nn.Module):
    def __init__(self, n_embed: int, max_seq_len: int) -> None:
        super().__init__()

        position_id = torch.arange(0, max_seq_len).unsqueeze(1)
        frequencies = torch.arange(0, n_embed, 2, dtype=torch.float32) / n_embed
        frequencies = torch.pow(10000.0, -frequencies)

        positional_encodings = torch.zeros(size=(max_seq_len, n_embed))
        # print(frequencies.shape, position_id.shape, positional_encodings.shape)

        positional_encodings[:, 0::2] = torch.sin(position_id * frequencies)
        positional_encodings[:, 1::2] = torch.cos(position_id * frequencies)

        self.register_buffer("positional_encodings", positional_encodings)

        self.dropout = nn.Dropout(p=GPTConfig.dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        pos_encodings = self.positional_encodings[: x.shape[1]]
        return self.dropout(pos_encodings + x)


# ------------------------------ NanoGPT Module ------------------------------ #
class NanoGPT(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        n_embed: int,
        n_heads: int,
        buffer_size: int,
        n_blocks: int,
    ) -> None:
        super().__init__()
        self.vocab_size = vocab_size
        self.n_embed = n_embed
        self.n_heads = n_heads
        self.buffer_size = buffer_size
        self.n_blocks = n_blocks

        self.token_embeddings = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=n_embed
        )

        if GPTConfig.use_sinusoidal:
            self.positional_encodings = PositionalEncoding(
                n_embed=n_embed, max_seq_len=GPTConfig.buffer_size
            )
        else:
            self.positional_encodings = nn.Embedding(
                num_embeddings=GPTConfig.buffer_size, embedding_dim=n_embed
            )

        self.blocks = nn.Sequential(
            *[AttentionBlock(n_embed=n_embed, n_heads=n_heads) for _ in range(n_blocks)]
        )
        self.ln = nn.LayerNorm((n_embed,))
        self.lm_head = nn.Sequential(
            nn.Linear(n_embed, n_embed // 2), nn.GELU(), nn.Linear(n_embed//2, vocab_size)
        )

    def forward(
        self, input_ids: torch.Tensor, labels: torch.Tensor = None
    ) -> torch.Tensor:
        B, T = input_ids.shape
        tok_emb = self.token_embeddings(input_ids) # (B, T, C)
        if GPTConfig.use_sinusoidal:
            x = self.positional_encodings.forward(tok_emb) # (B, T, C) -> (B, T, C)
        else:
            x = tok_emb + self.positional_encodings(
                torch.arange(T, dtype=torch.long, device=input_ids.shape)
            ) # (B, T, C) -> (B, T, C)
        x = self.blocks(x)
        x = self.ln(x)
        x = self.lm_head(x)

        loss = None
        if labels is not None:
            B, T, C = x.shape
            loss = F.cross_entropy(x.view(B * T, C), labels.view(B * T))

        return x, loss

    def generate(
        self, ids: torch.Tensor, max_len: int, temperature: float = 0.7
    ) -> int:
        for _ in range(max_len):
            ids_cond = ids[:, -GPTConfig.buffer_size :]
            logits, _ = self.forward(input_ids=ids_cond, labels=None)
            logit = logits[:, -1, :]
            probs = F.softmax(logit, dim=-1)
            val, idx = torch.topk(probs, k=int(probs.shape[1] * temperature), dim=-1)
            # print(val[0][0])
            idx_next = torch.multinomial(val, num_samples=1)
            idx_next = idx[:, idx_next][0]
            ids = torch.cat([ids, idx_next], dim=-1)
            if idx_next == 0:
                break
        return ids


if __name__ == "__main__":
    model = NanoGPT(
        vocab_size=GPTConfig.vocab_size,
        n_embed=GPTConfig.n_embed,
        n_heads=GPTConfig.n_head,
        buffer_size=GPTConfig.buffer_size,
        n_blocks=GPTConfig.n_layers,
    )

    print(model)

NanoGPT(
  (token_embeddings): Embedding(162, 368)
  (positional_encodings): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (blocks): Sequential(
    (0): AttentionBlock(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Attention(
            (Q): Linear(in_features=368, out_features=92, bias=False)
            (K): Linear(in_features=368, out_features=92, bias=False)
            (V): Linear(in_features=368, out_features=92, bias=False)
          )
        )
        (proj): Linear(in_features=368, out_features=368, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=368, out_features=1472, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1472, out_features=368, bias=True)
        )
      )
      (ln1): LayerNorm((368,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((368,), eps=1

### Engine funcs

In [9]:
def train_fn(net: NanoGPT, trainloader: torch.utils.data.DataLoader, epochs: int):
    loss_fct = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)
    net.train()

    for epoch in range(epochs):
        # print('#'*15)
        # print(f'### Epoch {epoch+1}/{epochs}')
        # print('#'*15)

        # pbar = tqdm(trainloader, total=len(trainloader), desc='(train)')
        correct, total, running_loss = 0, 0, 0.0

        for batch in trainloader:
            X, y = batch['input_ids'], batch['labels']
            X, y = X.to(DEVICE), y.to(DEVICE)

            optimizer.zero_grad()
            yHat, loss = net.forward(input_ids=X, labels=y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            total += y.size(0)
#             correct += (torch.argmax(yHat.data, dim=1) == y).sum().item()

            epoch_loss = running_loss / total
            # epoch_accuracy = correct / total

#             pbar.set_postfix({
#                 'loss': f'{epoch_loss:.4f}',
# #                 'accuracy': f'{epoch_accuracy*100:.4f}%'
#             })

        print(f'Epoch {epoch+1}: train loss {epoch_loss}')


@torch.no_grad()
def eval_fn(net: NanoGPT, testloader: torch.utils.data.DataLoader):
    loss_fct = nn.CrossEntropyLoss()
    correct, total, running_loss = 0., 0., 0.
    net.eval()

    # pbar = tqdm(testloader, total=len(testloader), desc='(eval)')

    for batch in testloader:
        X, y = batch['input_ids'], batch['labels']
        X, y = X.to(DEVICE), y.to(DEVICE)

        yHat, loss = net.forward(input_ids=X, labels=y)

        running_loss += loss.item()
        total += y.size(0)
#         correct += (torch.argmax(yHat.data, dim=1) == y).sum().item()

        epoch_loss = running_loss / total
#         epoch_accuracy = correct / total

#         pbar.set_postfix({
#             'loss': f'{epoch_loss:.4f}',
# #             'accuracy': f'{epoch_accuracy*100:.4f}%'
#         })

    return epoch_loss#, epoch_accuracy

In [10]:
trainloader = trainloaders[0]
valloader = validloaders[0]
net = NanoGPT(
    vocab_size=GPTConfig.vocab_size,
    n_embed=GPTConfig.n_embed,
    n_heads=GPTConfig.n_head,
    buffer_size=GPTConfig.buffer_size,
    n_blocks=GPTConfig.n_layers,
).to(DEVICE)

for epoch in range(5):
    train_fn(net, trainloader, 1)
    loss = eval_fn(net, valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}")

loss = eval_fn(net, testloader)
print(f"Final test set performance:\n\tloss {loss}")

Epoch 1: train loss 0.04706170739950957
Epoch 1: validation loss 0.03903460184733073
Epoch 1: train loss 0.03769006145221216
Epoch 2: validation loss 0.03686038891474406
Epoch 1: train loss 0.03551535078772792
Epoch 3: validation loss 0.034393905500570934
Epoch 1: train loss 0.03345601029970028
Epoch 4: validation loss 0.032225391070048015
Epoch 1: train loss 0.031660262942314145
Epoch 5: validation loss 0.030314618150393168
Final test set performance:
	loss 0.030108210704296066


## Step 2: Federated Learning

### Updating model parameters

In [11]:
def get_parameters(net: NanoGPT) -> List[np.array]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net: NanoGPT, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k,v in params_dict})
    net.load_state_dict(state_dict, strict=True)

### Client

In [12]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid: str, net: NanoGPT, trainloader: torch.utils.data.DataLoader, validloader: torch.utils.data.DataLoader) -> None:
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.validloader = validloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train_fn(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate: config: {config}")
        set_parameters(self.net, parameters)
        loss = eval_fn(self.net, self.validloader)
        return float(loss), len(self.validloader), {"loss": float(loss)}

### Virtual Client Engine

In [13]:
def client_fn(cid: str) -> FlowerClient:
    net = NanoGPT(
        vocab_size=GPTConfig.vocab_size,
        n_embed=GPTConfig.n_embed,
        n_heads=GPTConfig.n_head,
        buffer_size=GPTConfig.buffer_size,
        n_blocks=GPTConfig.n_layers,
    ).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    validloader = validloaders[int(cid)]
    return FlowerClient(cid=cid, net=net, trainloader=trainloader, validloader=validloader)

## Starting training

### Server Side Evaluation

In [14]:
def evaluate_fn(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar]
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = NanoGPT(
        vocab_size=GPTConfig.vocab_size,
        n_embed=GPTConfig.n_embed,
        n_heads=GPTConfig.n_head,
        buffer_size=GPTConfig.buffer_size,
        n_blocks=GPTConfig.n_layers,
    ).to(DEVICE)
    set_parameters(net, parameters)
    loss = eval_fn(net, testloader=testloader)
    print(f"\n\n>>> Server-side evaluation loss {loss}\n\n")
    # del net
    return loss, {"loss": loss}

In [15]:
def fit_config_fn(server_round: int):
    """Return training config dict for each round.

    Perform two rounds of training with one local epoch, increase two local
    epochs afterwards
    """
    config = {
        "server_round": server_round,
        "local_epochs": 2 if server_round < 2 else 3
    }
    return config

In [16]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,              # Sample 100% of the available clients for training
    fraction_evaluate=0.5,         # Sample 50% of available clients for eval
    min_fit_clients=NUM_CLIENTS,            # Never sample less than 10 clients for training
    min_evaluate_clients=NUM_CLIENTS,        # Never sample less than 5 clients for eval
    min_available_clients=NUM_CLIENTS,      # Wait until all 10 clients are available
    evaluate_fn=evaluate_fn,       # Evaluate function after every round
    on_fit_config_fn=fit_config_fn # Fit Config
)

client_resources = None
if DEVICE.type == "cuda":
    import multiprocessing
    client_resources = {"num_gpus": 1, "num_cpus": multiprocessing.cpu_count()}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources
)

INFO flwr 2023-11-08 15:37:54,486 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-11-08 15:37:57,160	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-11-08 15:37:59,759 | app.py:210 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7825924916.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3912962457.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7825924916.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3912962457.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-11-08 15:37:59,766 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_gpus': 1, 'num_cpus': 2}
INFO:flwr:Flower VCE: Resources for each Virtual Client: {'num_gpus': 1, 'num_cpus': 2}
INFO flwr 2023-11-08 15:37:59,829 | app.py:270 | Flo

(DefaultActor pid=21665) [Client 1] get_parameters


INFO flwr 2023-11-08 15:38:34,053 | server.py:94 | initial parameters (loss, other metrics): 0.15957860561249115, {'loss': 0.15957860561249115}
INFO:flwr:initial parameters (loss, other metrics): 0.15957860561249115, {'loss': 0.15957860561249115}
INFO flwr 2023-11-08 15:38:34,061 | server.py:104 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-11-08 15:38:34,065 | server.py:222 | fit_round 1: strategy sampled 3 clients (out of 3)
DEBUG:flwr:fit_round 1: strategy sampled 3 clients (out of 3)




>>> Server-side evaluation loss 0.15957860561249115


(DefaultActor pid=21665) [Client 1, round 1] fit, config: {'server_round': 1, 'local_epochs': 2}
(DefaultActor pid=21665) Epoch 1: train loss 0.04636669856530649
(DefaultActor pid=21665) Epoch 2: train loss 0.03813780644425639
(DefaultActor pid=21665) [Client 2, round 1] fit, config: {'server_round': 1, 'local_epochs': 2}
(DefaultActor pid=21665) Epoch 1: train loss 0.04545743770069546
(DefaultActor pid=21665) Epoch 2: train loss 0.03756315581224583
(DefaultActor pid=21665) [Client 0, round 1] fit, config: {'server_round': 1, 'local_epochs': 2}
(DefaultActor pid=21665) Epoch 1: train loss 0.04547571611625177


DEBUG flwr 2023-11-08 15:40:26,344 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures


...

(DefaultActor pid=21665) Epoch 3: train loss 0.02243731755901266


INFO flwr 2023-11-08 15:54:03,392 | server.py:125 | fit progress: (5, 0.021384137905731085, {'loss': 0.021384137905731085}, 929.3271177830002)
INFO:flwr:fit progress: (5, 0.021384137905731085, {'loss': 0.021384137905731085}, 929.3271177830002)
DEBUG flwr 2023-11-08 15:54:03,398 | server.py:173 | evaluate_round 5: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 5: strategy sampled 3 clients (out of 3)




>>> Server-side evaluation loss 0.021384137905731085


(DefaultActor pid=21665) [Client 2] evaluate: config: {}
(DefaultActor pid=21665) [Client 1] evaluate: config: {}
(DefaultActor pid=21665) [Client 0] evaluate: config: {}


DEBUG flwr 2023-11-08 15:54:06,109 | server.py:187 | evaluate_round 5 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 5 received 3 results and 0 failures
INFO flwr 2023-11-08 15:54:06,115 | server.py:153 | FL finished in 932.050093367
INFO:flwr:FL finished in 932.050093367
INFO flwr 2023-11-08 15:54:06,119 | app.py:225 | app_fit: losses_distributed [(1, 0.03643145759900411), (2, 0.029163284964031647), (3, 0.02533520387278663), (4, 0.023057628704441915), (5, 0.021657457815276247)]
INFO:flwr:app_fit: losses_distributed [(1, 0.03643145759900411), (2, 0.029163284964031647), (3, 0.02533520387278663), (4, 0.023057628704441915), (5, 0.021657457815276247)]
INFO flwr 2023-11-08 15:54:06,126 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-08 15:54:06,127 | app.py:227 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-11-08 15:54:06,129 | app.py:228 | app_fit: losses_centralized 

History (loss, distributed):
	round 1: 0.03643145759900411
	round 2: 0.029163284964031647
	round 3: 0.02533520387278663
	round 4: 0.023057628704441915
	round 5: 0.021657457815276247
History (loss, centralized):
	round 0: 0.15957860561249115
	round 1: 0.03604011293031571
	round 2: 0.028900699646800896
	round 3: 0.0250554058386831
	round 4: 0.02278480763537893
	round 5: 0.021384137905731085
History (metrics, centralized):
{'loss': [(0, 0.15957860561249115), (1, 0.03604011293031571), (2, 0.028900699646800896), (3, 0.0250554058386831), (4, 0.02278480763537893), (5, 0.021384137905731085)]}